# Install libraries and import dependencies

In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!pip install geonamescache

In [15]:
!python -m spacy download en_core_web_lg

2023-11-25 10:40:10.880666: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-25 10:40:10.880742: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-25 10:40:10.880784: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-25 10:40:12.193003: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 3.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [16]:
!pip show spacy

Name: spacy
Version: 3.7.2
Summary: Industrial-strength Natural Language Processing (NLP) in Python
Home-page: https://spacy.io
Author: Explosion
Author-email: contact@explosion.ai
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: catalogue, cymem, jinja2, langcodes, murmurhash, numpy, packaging, preshed, pydantic, requests, setuptools, smart-open, spacy-legacy, spacy-loggers, srsly, thinc, tqdm, typer, wasabi, weasel
Required-by: en-core-web-lg, en-core-web-sm, fastai


In [17]:
import re
import os, sys, email
import pandas as pd
import numpy as np

import spacy
import geonamescache

In [34]:
# Load and use a trained model
nlp = spacy.load("en_core_web_lg")
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

# Functions

In [35]:
def gen_dict_extract(var, key):
    if isinstance(var, dict):
        for k, v in var.items():
            if k == key:
                yield v
            if isinstance(v, (dict, list)):
                yield from gen_dict_extract(v, key)
    elif isinstance(var, list):
        for d in var:
            yield from gen_dict_extract(d, key)

In [36]:
def spacy_ner_extratction(text):
    global df_sc_name, df_sc_org, df_sc_date, df_sc_city, df_sc_state, df_sc_country, df_sc_cardinal

    gc = geonamescache.GeonamesCache()
    # gets nested dictionary for countries
    countries = gc.get_countries()
    # gets nested dictionary for cities
    cities = gc.get_cities()
    cities = [*gen_dict_extract(cities, 'name')]
    countries = [*gen_dict_extract(countries, 'name')]

    nlp = spacy.load("en_core_web_lg")

    sc_name = list()
    sc_org = list()
    sc_date = list()
    sc_city = list()
    sc_state = list()
    sc_country = list()
    sc_cardinal = list()

    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == 'PERSON':
            sc_name.append(ent.text)
        if ent.label_ == 'CARDINAL':
          sc_cardinal.append(ent.text)
        if ent.label_ == 'ORG':
            sc_org.append(ent.text)
        if ent.label_ == 'DATE':
            sc_date.append(ent.text)
        if ent.label_ == 'GPE':
            if ent.text in countries:
                sc_country.append(ent.text)
            elif ent.text in cities:
                sc_city.append(ent.text)
            else:
                sc_state.append(ent.text)

    df_sc_name = pd.DataFrame()
    df_sc_org = pd.DataFrame()
    df_sc_date = pd.DataFrame()
    df_sc_city = pd.DataFrame()
    df_sc_state = pd.DataFrame()
    df_sc_country = pd.DataFrame()
    df_sc_cardinal = pd.DataFrame()

    # Remove duplicates
#    set_sc_city = list(set(sc_city))
#    set_sc_contry = list(set(sc_contry))

    df_sc_name["Name"] = sc_name
    df_sc_org["Organization"] = sc_org
    df_sc_date["Date"] = sc_date
    df_sc_city["City"] = sc_city
    df_sc_state["State"] = sc_state
    df_sc_country["Country"] = sc_country
    df_sc_cardinal["Cardinal"] = sc_cardinal


In [37]:
def get_email_addresses(text):
    global get_email, df_get_email, string, email_ori, email, remove_id, email_set, domain_name, final_email_list, remove_count, remove_email, set_remove_email

    get_email = list()
    email_set = list()
    remove_id = list()
    remove_email = list()
    id_1 = id_2 = id_3 = id_4 = id_5 = id_6 = remove_count = 0

    r = re.compile(r'[\w\.-]+@[\w\.-]+')

    string = text
    #string = string.replace('\n', '<br />')
    email_ori = r.findall(string)
    email = email_ori.copy()

    for id_1 in range(len(email)):
        email[id_1] = email[id_1].lower()

        if email[id_1][-1] == ".":
            email[id_1] = email[id_1][:-1]

    df_get_email = pd.DataFrame(columns = ["Email"])
    for id in range(len(email)):
        df_get_email["Email"] = email

# Process

In [38]:
text = """
SM ABDULLAH AL NOMAAN
Niloy 29/Jha, BAF Officers’ Quarters, Dhaka Cantonment, Bangladesh
Mobile: +8801869518002
Email: nomaan8002@gmail.com
NID: 5552193020
OBJECTIVE: Ambitious and driven to secure a challenging and rewarding position as
Business Professional to increase productivity and quality of service to fulfill the company’s
strategic requirements.

CAREER SUMMARY
▪ PS to Managing Director at Crony Group
(March 2023- Till date)
• Proved successful working within tight deadlines and a fast-paced environment.
• Assisted with day-to-day operations, working efficiently and productively.
• Participated in team projects to work collaboratively and effectively.
• Strengthened communication skills through regular interactions with others.
• Identified issues, analyzed information and provided solutions to problems.
▪ HR Executive Officer at DataPath Ltd.
(February 2021- January 2023)
• Work with an end-to-end recruitment process.
• Communicate professionally with clients, vendors, and co-workers.
• HR systems update time to time.
• Organizing events with the team.
• Manage employee files.
• Overseeing the administrative needs on a daily basis.
• Research and Development for the branding of the Company.
• Prepare New Hire Paperwork and Termination Paperwork for SSAE16 Audit.
▪ Internship Program under HR Department
At DataPath Ltd.
▪Quality Assurance Associate & Event Organizer
At SHETRADES

(September 2020- January 2021)
(October 2019-February 2020)

Training and Development
•

Professional Certificate in HR Operations from World Academy for Research
Development in 2022

EDUCATIONAL QUALIFICATIONS:
American International University-Bangladesh

2021

Bachelor of Business Administration
1st Major in Management Information System
2nd Major in Human Resource Management
Current CGPA: 3.63/4.00
BAF Shaheen College Dhaka Bangladesh

2016

Higher Secondary Certificate from Humanities
GPA: 3.42/5
BAF Shaheen School Dhaka Bangladesh

2014

Secondary School Certificate from Humanities
GPA: 4.25/5

INTERESTS:
IOT based solution development, developing web-based solution through inclusive
Management Information System for Business Development.

HONOURS AND AWARDS:
• Champion of MIS poster contest summer 2019.
• Champion of poster contest in CS FEST 2018.
• Power of people poster presentation 2nd runner up under HRM Dept. 2017.

SKILLS:
• Fluency in both Bengali and English languages.
• Microsoft word, Microsoft excel, MS PowerPoint, Focusky, Orange HR, WordPress,
Adobe flash, Photoshop, Illustrator, DGEM, Unify

REFERENCES:
Khandaker Fazle Rabbi
Director, Human Resource Department
DataPath Limited.
Mobile: 01923746476

Mehzab H.Nahid
Assistant Professor, Dept of MIS
American International University-Bangladesh (AIUB)
Mobile: 01777787316
 """

In [39]:
# Extract names, organizations, date, cities, states, and countries:
spacy_ner_extratction(text)
# Extract emails
get_email_addresses(text)

In [40]:
# Merge all the columns into a dataframe:
df_output = pd.concat([df_sc_name, df_get_email, df_sc_org, df_sc_date, df_sc_city, df_sc_state, df_sc_country, df_sc_cardinal], axis=1)

In [41]:
df_output

,Name,Email,Organization,Date,City,State,Country,Cardinal
0,HONOURS,nomaan8002@gmail.com,SM ABDULLAH,March 2023-,Mobile,Dhaka Cantonment,Bangladesh,5552193020
1,Khandaker Fazle Rabbi,NaN,BAF Officers’ Quarters,February 2021- January 2023,Mobile,Dhaka Bangladesh,NaN,3.63/4.00
2,NaN,NaN,NID,daily,Mobile,Dhaka Bangladesh,NaN,4.25/5
3,NaN,NaN,Business Professional,September 2020- January 2021,NaN,Focusky,NaN,01923746476
4,NaN,NaN,PS,October 2019-February 2020,NaN,NaN,NaN,01777787316
5,NaN,NaN,Crony Group,2022,NaN,NaN,NaN,NaN
6,NaN,NaN,DataPath Ltd.,2021,NaN,NaN,NaN,NaN
7,NaN,NaN,Company,2016,NaN,NaN,NaN,NaN
8,NaN,NaN,Internship Program,3.42/5,NaN,NaN,NaN,NaN
9,NaN,NaN,HR Department,2014,NaN,NaN,NaN,NaN
